In [ ]:
from recbole.quick_start import load_data_and_model
from recbole.trainer import Trainer

import recbole.evaluator.register
import recbole.evaluator.collector
import copy
import torch

# monkey patch: 修复evaluator的bug: 'Counter' object has no attribute 'cpu'
new_Collector = recbole.evaluator.collector.Collector
def get_data_struct_new(self):
    """Get all the evaluation resource that been collected.
    And reset some of outdated resource.
    """
    for key in self.data_struct._data_dict:
        if isinstance(self.data_struct._data_dict[key], torch.Tensor):
            self.data_struct._data_dict[key] = self.data_struct._data_dict[key].cpu()
        else:
            self.data_struct._data_dict[key] = self.data_struct._data_dict[key] 
    returned_struct = copy.deepcopy(self.data_struct)
    for key in ["rec.topk", "rec.meanrank", "rec.score", "rec.items", "data.label"]:
        if key in self.data_struct:
            del self.data_struct[key]
    return returned_struct

new_Collector.get_data_struct = get_data_struct_new


config, model, dataset, train_data, valid_data, test_data = load_data_and_model('saved/yourfile')

# 调整config参数
config['topk'] = [10, 20, 50]
config['metrics'] = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'map']

trainer = Trainer(config, model)
test_result = trainer.evaluate(model_file='saved/yourfile', eval_data=test_data)  # 输出测试集指标
print(test_result)

06 Jun 22:47    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path

 = dataset/amazon-books-23
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 5000
train_batch_size = 8192
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'none', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'map', 'averagepopularity']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 8192
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIE

OrderedDict([('recall@10', 0.0955), ('recall@20', 0.147), ('recall@50', 0.2412), ('mrr@10', 0.1053), ('mrr@20', 0.1124), ('mrr@50', 0.1174), ('ndcg@10', 0.0715), ('ndcg@20', 0.0888), ('ndcg@50', 0.1152), ('hit@10', 0.2496), ('hit@20', 0.353), ('hit@50', 0.5064), ('precision@10', 0.0325), ('precision@20', 0.0254), ('precision@50', 0.0171), ('map@10', 0.0413), ('map@20', 0.0456), ('map@50', 0.0501)])
